# Intro to DataFrames

References:

- DataFrame API docs: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

Rules of thumb:
- Hit tab to auto-complete
- To see all available methods, place a dot (.) after the RDD (e.g. words.) and hit tab 
- Use `.collect()` to see the contents of the RDD

Solutions for potentially challenging exercises can be found in the end of the section. Don't peek unless you're really stuck!

In [2]:
# like in the pyspark shell, SparkSession is already defined
spark

## 1. DataFrame methods

### 1.1 Data input

In [2]:
df = spark.read.json("../data/people/names.json")

# other supported file formats:
# spark.read.parquet("../data/pems_sorted/")
# spark.read.text()
# spark.read.csv()
# spark.read.orc()

# generic form: 
# spark.read.load("path/to/someFile.csv", format="csv", sep=":", inferSchema="true", header="true")

# Loading data from a JDBC source
# jdbcDF = spark.read \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql:dbserver") \
#     .option("dbtable", "schema.tablename") \
#     .option("user", "username") \
#     .option("password", "password") \
#     .load()

In [4]:
# TODO: write reading different files in ../data

In [5]:
spark.read.parquet("../data/pems_sorted/")

DataFrame[timeperiod: string, flow1: int, occupancy1: double, speed1: double, flow2: int, occupancy2: double, speed2: double, flow3: int, occupancy3: double, speed3: double, flow4: int, occupancy4: double, speed4: double, flow5: int, occupancy5: double, speed5: double, flow6: int, occupancy6: double, speed6: double, flow7: int, occupancy7: double, speed7: double, flow8: int, occupancy8: double, speed8: double, station: int]

In [6]:
spark.read.csv("../data/mobile_data/MobileSampleData.csv", header=True).show(3)

+--------+---------+------+--------------+----------+-----------+------------+-------------+---------+--------------------+
|ClientID|QueryTime|Market|DevicePlatform|DeviceMake|DeviceModel|       State|      Country|SessionId|SessionPageViewOrder|
+--------+---------+------+--------------+----------+-----------+------------+-------------+---------+--------------------+
|       8| 18:54:20| en-US|       Android|   Samsung|   SCH-i500|  California|United States|        0|                   0|
|      23| 19:19:44| en-US|       Android|       HTC| Incredible|Pennsylvania|United States|        0|                   0|
|      23| 19:19:46| en-US|       Android|       HTC| Incredible|Pennsylvania|United States|        0|                   1|
+--------+---------+------+--------------+----------+-----------+------------+-------------+---------+--------------------+
only showing top 3 rows



### 1.2 Data output (writing to disk)

- API docs: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter

In [7]:
# writing to a file
df.write.parquet("new_data.parquet")

In [8]:
# overwrite on save
df.write.mode("overwrite").parquet("new_data.parquet")

In [9]:
# You can read from any format and write to any format (barring formatting limitations/rules):
# df.write.csv("new_data.csv",header=True)
# df.write.json("new_data.json")
# df.write.orc("new_data.orc")
# df.write.parquet("new_data.parquet")

# generic form:
# df.write.save("fileName.parquet", format="parquet")
# df.write.mode("overwrite").save("fileName.parquet", format="parquet")

# Saving data to a JDBC source
# jdbcDF.write \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql:dbserver") \
#     .option("dbtable", "schema.tablename") \
#     .option("user", "username") \
#     .option("password", "password") \
#     .save()


### Exploring DataFrames

In [10]:
df = spark.read.json("../data/people/names.json")

In [11]:
df.head(5)

[Row(AGE=36, gender='female', height=180, name='Zoe'),
 Row(AGE=23, gender='female', height=165, name='Alice'),
 Row(AGE=30, gender='male', height=175, name='Andy'),
 Row(AGE=25, gender='female', height=170, name='Jane'),
 Row(AGE=None, gender='male', height=165, name='Michael')]

In [12]:
df.show(5)

+----+------+------+-------+
| AGE|gender|height|   name|
+----+------+------+-------+
|  36|female|   180|    Zoe|
|  23|female|   165|  Alice|
|  30|  male|   175|   Andy|
|  25|female|   170|   Jane|
|null|  male|   165|Michael|
+----+------+------+-------+
only showing top 5 rows



In [13]:
df.take(5)

[Row(AGE=36, gender='female', height=180, name='Zoe'),
 Row(AGE=23, gender='female', height=165, name='Alice'),
 Row(AGE=30, gender='male', height=175, name='Andy'),
 Row(AGE=25, gender='female', height=170, name='Jane'),
 Row(AGE=None, gender='male', height=165, name='Michael')]

In [14]:
# limit(n) returns a new dataframe with the first n rows of the dataframe
df.limit(3).show()

+---+------+------+-----+
|AGE|gender|height| name|
+---+------+------+-----+
| 36|female|   180|  Zoe|
| 23|female|   165|Alice|
| 30|  male|   175| Andy|
+---+------+------+-----+



In [15]:
df.show()

+----+------+------+-------+
| AGE|gender|height|   name|
+----+------+------+-------+
|  36|female|   180|    Zoe|
|  23|female|   165|  Alice|
|  30|  male|   175|   Andy|
|  25|female|   170|   Jane|
|null|  male|   165|Michael|
|  19|  male|   180| Justin|
+----+------+------+-------+



In [16]:
df.printSchema()

root
 |-- AGE: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- height: long (nullable = true)
 |-- name: string (nullable = true)



In [17]:
df.columns

['AGE', 'gender', 'height', 'name']

In [18]:
df.count()

6

In [19]:
df.describe().show()

+-------+-----------------+------+-----------------+-----+
|summary|              AGE|gender|           height| name|
+-------+-----------------+------+-----------------+-----+
|  count|                5|     6|                6|    6|
|   mean|             26.6|  null|            172.5| null|
| stddev|6.580273550544841|  null|6.892024376045112| null|
|    min|               19|female|              165|Alice|
|    max|               36|  male|              180|  Zoe|
+-------+-----------------+------+-----------------+-----+



### Selecting specific columns in a dataframe

In [20]:
# selecting a column
# Column API docs: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column
df['name']



Column<b'name'>

In [21]:
# creating a new dataframe with only selected columns
df.select('name')

DataFrame[name: string]

In [22]:
# creating a new dataframe with only selected columns
df.select(['name', 'age']).show()

+-------+----+
|   name| age|
+-------+----+
|    Zoe|  36|
|  Alice|  23|
|   Andy|  30|
|   Jane|  25|
|Michael|null|
| Justin|  19|
+-------+----+



In [23]:
# renaming columns
df = df.withColumnRenamed('AGE', 'age')

In [24]:
df

DataFrame[age: bigint, gender: string, height: bigint, name: string]

In [25]:
# Creating new columns
df = df.withColumn('height plus 100', df.height + 100)
df.show()

+----+------+------+-------+---------------+
| age|gender|height|   name|height plus 100|
+----+------+------+-------+---------------+
|  36|female|   180|    Zoe|            280|
|  23|female|   165|  Alice|            265|
|  30|  male|   175|   Andy|            275|
|  25|female|   170|   Jane|            270|
|null|  male|   165|Michael|            265|
|  19|  male|   180| Justin|            280|
+----+------+------+-------+---------------+



In [26]:
# Creating new columns
df = df.withColumn('is_tall', df.height >= 175)
df.show()

+----+------+------+-------+---------------+-------+
| age|gender|height|   name|height plus 100|is_tall|
+----+------+------+-------+---------------+-------+
|  36|female|   180|    Zoe|            280|   true|
|  23|female|   165|  Alice|            265|  false|
|  30|  male|   175|   Andy|            275|   true|
|  25|female|   170|   Jane|            270|  false|
|null|  male|   165|Michael|            265|  false|
|  19|  male|   180| Justin|            280|   true|
+----+------+------+-------+---------------+-------+



### Filtering
`.filter()` takes in either (i) a `Column` of `types.BooleanType` or (ii) a string of SQL expression.

In [27]:
# filter using SQL expressions
# df.where('age >= 25').show() is also possible because .where() is an alias for .filter()
df.filter('age >= 25').show()

+---+------+------+----+---------------+-------+
|age|gender|height|name|height plus 100|is_tall|
+---+------+------+----+---------------+-------+
| 36|female|   180| Zoe|            280|   true|
| 30|  male|   175|Andy|            275|   true|
| 25|female|   170|Jane|            270|  false|
+---+------+------+----+---------------+-------+



In [28]:
# filter using a column of boolean types
df.filter(df.age >= 25).show()

+---+------+------+----+---------------+-------+
|age|gender|height|name|height plus 100|is_tall|
+---+------+------+----+---------------+-------+
| 36|female|   180| Zoe|            280|   true|
| 30|  male|   175|Andy|            275|   true|
| 25|female|   170|Jane|            270|  false|
+---+------+------+----+---------------+-------+



In [29]:
# df.age >= 25 returns a Column of booleans
df.age >= 25

Column<b'(age >= 25)'>

In [30]:
df.filter( (df.age >= 25) & (df.age <= 30) ).show()
# you can use df.age or df['age']
# you can replace & with | for 'or' operations

+---+------+------+----+---------------+-------+
|age|gender|height|name|height plus 100|is_tall|
+---+------+------+----+---------------+-------+
| 30|  male|   175|Andy|            275|   true|
| 25|female|   170|Jane|            270|  false|
+---+------+------+----+---------------+-------+



### groupBy

TL;DR - `.groupBy()` allows you to group rows together based on its value in some given column(s)
- `df.groupBy([cols])`
- [GroupedData operations](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) (alternatively, you can instantiate a variable with the type of GroupedData, let jupyter notebook's intellisense show you what methods are available:
    - `grouped = df.groupBy('gender')`
    - `grouped.` (and hit tab)

In [31]:
df.head()

Row(age=36, gender='female', height=180, name='Zoe', height plus 100=280, is_tall=True)

In [32]:
df.groupBy('gender')

In [33]:
grouped = df.groupBy('gender')

In [34]:
df.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|female|    3|
|  male|    3|
+------+-----+



In [35]:
df.groupBy('gender').avg().show()

+------+--------+------------------+--------------------+
|gender|avg(age)|       avg(height)|avg(height plus 100)|
+------+--------+------------------+--------------------+
|female|    28.0|171.66666666666666|   271.6666666666667|
|  male|    24.5|173.33333333333334|   273.3333333333333|
+------+--------+------------------+--------------------+



In [36]:
# calculate total age and height
df.groupBy().sum().show()

+--------+-----------+--------------------+
|sum(age)|sum(height)|sum(height plus 100)|
+--------+-----------+--------------------+
|     133|       1035|                1635|
+--------+-----------+--------------------+



In [37]:
# TODO: calculate average height of all people (i.e. don't groupby anything)

In [38]:
# TODO: calculate max height for each gender

In [39]:
# TODO: calculate min height for each gender

### 1.2 Crimes Data

In [40]:
crimes = spark.read.csv("../data/crimes/Crimes_-_One_year_prior_to_present.csv", header=True, inferSchema=True)
# try the above without the header and inferSchema option. see what happens!

In [41]:
# TODO: print the schema of the dataframe (e.g. data type of each column)?
crimes.printSchema()

root
 |-- CASE#: string (nullable = true)
 |-- DATE  OF OCCURRENCE: string (nullable = true)
 |-- BLOCK: string (nullable = true)
 |--  IUCR: string (nullable = true)
 |--  PRIMARY DESCRIPTION: string (nullable = true)
 |--  SECONDARY DESCRIPTION: string (nullable = true)
 |--  LOCATION DESCRIPTION: string (nullable = true)
 |-- ARREST: string (nullable = true)
 |-- DOMESTIC: string (nullable = true)
 |-- BEAT: integer (nullable = true)
 |-- WARD: integer (nullable = true)
 |-- FBI CD: string (nullable = true)
 |-- X COORDINATE: integer (nullable = true)
 |-- Y COORDINATE: integer (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)



In [42]:
# - how many rows are there in the dataframe?
crimes.count()

263191

In [43]:
# TODO: Display the first 2 rows
crimes.take(2)

[Row(CASE#='JB241987', DATE  OF OCCURRENCE='04/28/2018 10:05:00 PM', BLOCK='009XX N LONG AVE',  IUCR='2092',  PRIMARY DESCRIPTION='NARCOTICS',  SECONDARY DESCRIPTION='SOLICIT NARCOTICS ON PUBLICWAY',  LOCATION DESCRIPTION='SIDEWALK', ARREST='Y', DOMESTIC='N', BEAT=1524, WARD=37, FBI CD='18', X COORDINATE=1140136, Y COORDINATE=1905903, LATITUDE=41.897894893, LONGITUDE=-87.760743714, LOCATION='(41.897894893, -87.760743714)'),
 Row(CASE#='JA430240', DATE  OF OCCURRENCE='09/06/2017 01:30:00 PM', BLOCK='032XX W 26TH ST',  IUCR='0810',  PRIMARY DESCRIPTION='THEFT',  SECONDARY DESCRIPTION='OVER $500',  LOCATION DESCRIPTION='OTHER', ARREST='Y', DOMESTIC='N', BEAT=1024, WARD=12, FBI CD='06', X COORDINATE=1155313, Y COORDINATE=1886555, LATITUDE=41.844510467, LONGITUDE=-87.705519454, LOCATION='(41.844510467, -87.705519454)')]

In [44]:
# TODO: What columns are in the dataframe?
crimes.columns

['CASE#',
 'DATE  OF OCCURRENCE',
 'BLOCK',
 ' IUCR',
 ' PRIMARY DESCRIPTION',
 ' SECONDARY DESCRIPTION',
 ' LOCATION DESCRIPTION',
 'ARREST',
 'DOMESTIC',
 'BEAT',
 'WARD',
 'FBI CD',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION']

In [45]:
# Let's rename the improperly formatted column names
columnNames = crimes.columns
for col in columnNames:
    crimes = crimes.withColumnRenamed(col, col.strip())
    
crimes.columns

['CASE#',
 'DATE  OF OCCURRENCE',
 'BLOCK',
 'IUCR',
 'PRIMARY DESCRIPTION',
 'SECONDARY DESCRIPTION',
 'LOCATION DESCRIPTION',
 'ARREST',
 'DOMESTIC',
 'BEAT',
 'WARD',
 'FBI CD',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION']

In [46]:
# TODO: How many cases resulted in arrest, and how many didn’t?
# Hint: Highlight whitespace between this cell and the next cell to see the hint

crimes.groupBy("ARREST").count().show()

+------+------+
|ARREST| count|
+------+------+
|     Y| 50126|
|     N|213065|
+------+------+



<font color="white">Use .groupBy("ARREST")</font>

In [47]:
# TODO: List the total count of cases for each WARD
crimes.groupBy("WARD").count().show()

+----+-----+
|WARD|count|
+----+-----+
|  31| 3258|
|  34| 6845|
|  28|11071|
|  27|10000|
|  26| 3498|
|  44| 3968|
|  12| 3036|
|  22| 3014|
|  47| 2708|
|null|    1|
|   1| 4873|
|  13| 3086|
|  16| 6234|
|   6| 8427|
|   3| 6860|
|  20| 7608|
|  40| 2941|
|  48| 2641|
|   5| 6252|
|  19| 2207|
+----+-----+
only showing top 20 rows



In [48]:
# TODO: List the total count of cases for each WARD, and sort it (by count) in ascending order
crimes.groupBy("WARD").count().sort("count").show()

+----+-----+
|WARD|count|
+----+-----+
|null|    1|
|  19| 2207|
|  36| 2536|
|  39| 2598|
|  48| 2641|
|  33| 2698|
|  47| 2708|
|  38| 2738|
|  45| 2761|
|  50| 2877|
|  41| 2913|
|  40| 2941|
|  22| 3014|
|  12| 3036|
|  13| 3086|
|  14| 3146|
|  23| 3155|
|  35| 3181|
|  11| 3216|
|  30| 3249|
+----+-----+
only showing top 20 rows



In [49]:
# TODO: Show top 10 (WARD, count) pairs with the most number of cases
from pyspark.sql.functions import desc

crimes.groupBy("WARD").count().sort(desc("count")).show(10)

+----+-----+
|WARD|count|
+----+-----+
|  42|18548|
|  24|12457|
|   2|11483|
|  28|11071|
|  27|10000|
|  17| 8595|
|   6| 8427|
|  21| 8043|
|  20| 7608|
|   3| 6860|
+----+-----+
only showing top 10 rows



In [50]:
# TODO: List top 15 categories (PRIMARY DESCRIPTION) of cases
crimes.groupBy('PRIMARY DESCRIPTION').count().sort(desc("count")).show(15)

+--------------------+-----+
| PRIMARY DESCRIPTION|count|
+--------------------+-----+
|               THEFT|64285|
|             BATTERY|49276|
|     CRIMINAL DAMAGE|28118|
|             ASSAULT|19740|
|  DECEPTIVE PRACTICE|17923|
|       OTHER OFFENSE|16561|
|            BURGLARY|12040|
|           NARCOTICS|11664|
|             ROBBERY|11080|
| MOTOR VEHICLE THEFT|10558|
|   CRIMINAL TRESPASS| 6832|
|   WEAPONS VIOLATION| 5003|
|OFFENSE INVOLVING...| 2247|
| CRIM SEXUAL ASSAULT| 1539|
|PUBLIC PEACE VIOL...| 1386|
+--------------------+-----+
only showing top 15 rows



In [51]:
# TODO: List top 5 locations (LOCATION DESCRIPTION) where cases occur
crimes.groupBy('LOCATION DESCRIPTION').count().sort(desc("count")).show(5)

+--------------------+-----+
|LOCATION DESCRIPTION|count|
+--------------------+-----+
|              STREET|58758|
|           RESIDENCE|43732|
|           APARTMENT|33277|
|            SIDEWALK|20542|
|               OTHER|10791|
+--------------------+-----+
only showing top 5 rows



In [52]:
# TODO: Save one of the results to disk (choose any format)
# Note: if your dataframe ends up being partitioned, you can call `your_df.coalesce(1)` before saving (`df.coalesce(1).write...`)
top_locations = crimes.groupBy('LOCATION DESCRIPTION').count().sort(desc("count")).limit(5)
top_locations.coalesce(1).write.mode("overwrite").csv("../data/top_locations", header=True)

In [53]:
# TODO: submit the preceeding task as a spark job
# 1. Create a python file named jobs/top_20_crime_locations.py
# 2. define spark session object
#   - from pyspark.sql import SparkSession
#   - spark = SparkSession.builder.appName("MyAppName").getOrCreate()
# 3. Copy the code in the preceeding cell into the file 
# 4. submit the job: ${SPARK_HOME}/bin/spark-submit --master local ./jobs/top_20_crime_locations.py

# if you get stuck, you can refer to ./jobs/top_N_crime_locations_solution.py

In [54]:
# TODO: Use your creativity - create any other interesting DataFrames or insights into the crimes data!

## Using SQL with Spark DataFrames

In [55]:
df = spark.read.json("../data/people/names.json")

In [56]:
df.createOrReplaceTempView('names')

In [57]:
spark.sql("SELECT * FROM names")
# add .show() to see the resulting dataframe. Example:
# df = spark.sql("SELECT * FROM names")
# df.show()

DataFrame[AGE: bigint, gender: string, height: bigint, name: string]

In [58]:
spark.sql("SELECT * FROM names WHERE height > 170").show()

+---+------+------+------+
|AGE|gender|height|  name|
+---+------+------+------+
| 36|female|   180|   Zoe|
| 30|  male|   175|  Andy|
| 19|  male|   180|Justin|
+---+------+------+------+

